In [1]:
import pyarrow.parquet as pq
import matplotlib.pyplot as plt
import os
import pandas as pd
import tqdm
import pyarrow as pa

import warnings
warnings.filterwarnings('ignore')

plt.style.use('fivethirtyeight')

In [2]:
# Задаем данные
target_path = "train_target.csv"
path = 'train_data'

target_df = pd.read_csv(target_path)

print(f"Обработка файла завершена.")

Обработка файла завершена.


Все признаки, которые "бинаризированы" и "закодированы" имеют случайные числовые промежутки, поэтому к ним попробуем применить OneHotEncoding, а к признакам из binarized_columns просто агрегацию. 

In [3]:
def read_parquet_dataset_from_local(path_to_dataset: str, start_from: int = 0,
                                    num_parts_to_read: int = 2, columns=None, verbose=False) -> pd.DataFrame:
    """
    читает num_parts_to_read партиций, преобразовывает их к pd.DataFrame и возвращает
    :param path_to_dataset: путь до директории с партициями
    :param start_from: номер партиции, с которой нужно начать чтение
    :param num_parts_to_read: количество партиций, которые требуется прочитать
    :param columns: список колонок, которые нужно прочитать из партиции
    :return: pd.DataFrame
    """

    res = []
    dataset_paths = sorted([os.path.join(path_to_dataset, filename) for filename in os.listdir(path_to_dataset)
                            if filename.startswith('train')])
    print(dataset_paths)

    start_from = max(0, start_from)
    chunks = dataset_paths[start_from: start_from + num_parts_to_read]
    if verbose:
        print('Reading chunks:\n')
        for chunk in chunks:
            print(chunk)
    for chunk_path in tqdm.tqdm_notebook(chunks, desc="Reading dataset with pandas"):
        print('chunk_path', chunk_path)
        chunk = pd.read_parquet(chunk_path,columns=columns)
        res.append(chunk)

    return pd.concat(res).reset_index(drop=True)

In [4]:
def prepare_transactions_dataset(path_to_dataset: str, num_parts_to_preprocess_at_once: int = 1, num_parts_total: int=50,
                                 save_to_path=None, verbose: bool=False):
    """
    возвращает готовый pd.DataFrame с признаками, на которых можно учить модель для целевой задачи
    path_to_dataset: str
        путь до датасета с партициями
    num_parts_to_preprocess_at_once: int
        количество партиций, которые будут одновременно держаться и обрабатываться в памяти
    num_parts_total: int
        общее количество партиций, которые нужно обработать
    save_to_path: str
        путь до папки, в которой будет сохранён каждый обработанный блок в .parquet-формате; если None, то не будет сохранён
    verbose: bool
        логирует каждую обрабатываемую часть данных
    """
    preprocessed_frames = []

    for step in tqdm.tqdm_notebook(range(0, num_parts_total, num_parts_to_preprocess_at_once),
                                   desc="Transforming transactions data"):
        data_frame = read_parquet_dataset_from_local(path_to_dataset, step, num_parts_to_preprocess_at_once,
                                                             verbose=verbose)

        #здесь должен быть препроцессинг данных
        data_frame['no_delinquencies'] = (data_frame['is_zero_loans530'].astype(bool) & data_frame['is_zero_loans3060'].astype(bool) &
                                          data_frame['is_zero_loans90'].astype(bool) & data_frame['is_zero_loans6090'].astype(bool) & 
                                          data_frame['is_zero_loans5'].astype(bool)).astype(int)
        
        data_frame['total_delinquencies'] = data_frame['is_zero_loans530'] + data_frame['is_zero_loans3060'] + data_frame['is_zero_loans5'] + data_frame['is_zero_loans90'] + data_frame['is_zero_loans6090']
        
        binarized_columns = ['is_zero_loans5', 'is_zero_loans530', 'is_zero_loans3060', 'is_zero_loans6090', 'is_zero_loans90', 'is_zero_util', 'is_zero_over2limit', 'is_zero_maxover2limit', 'pclose_flag', 'fclose_flag', 'no_delinquencies']
        
        # Датафрейм содержащий в себе только действительно бинарные признаки
        df_second = data_frame[['id'] + binarized_columns].copy()
        
        # Датафрейм содержащий в себе все остальные признаки, которые пойдут в ohe
        df_first = data_frame.drop(binarized_columns, axis=1)
        
        feature_columns = list(df_first.columns.values)
        feature_columns.remove("id")
        feature_columns.remove("rn")
        
        # Кодируем датафрейм в ohe
        dummies = pd.get_dummies(df_first[feature_columns], columns=feature_columns)
        dummy_features = dummies.columns.values
        ohe_features = pd.concat([df_first, dummies], axis=1)
        ohe_features = ohe_features.drop(columns=feature_columns)
        
        # Группируем по id и sum данные после ohe
        ohe_features.groupby("id")
        features = ohe_features.groupby("id")[dummy_features].sum().reset_index(drop=False)
        
        # Здесь просто группируем и агрегируем 
        df_second = df_second.groupby('id').sum().reset_index(drop=False)
        
        # Объединяем датафреймы обратно
        df_result = pd.merge(features, df_second, on='id')
        
        #записываем подготовленные данные в файл
        preprocessed_frames.append(df_result)
    return pd.concat(preprocessed_frames, join='inner', ignore_index=True)

In [5]:
data = prepare_transactions_dataset(path, num_parts_to_preprocess_at_once=2, num_parts_total=12)

Transforming transactions data:   0%|          | 0/6 [00:00<?, ?it/s]

['train_data/train_data_0.pq', 'train_data/train_data_1.pq', 'train_data/train_data_10.pq', 'train_data/train_data_11.pq', 'train_data/train_data_2.pq', 'train_data/train_data_3.pq', 'train_data/train_data_4.pq', 'train_data/train_data_5.pq', 'train_data/train_data_6.pq', 'train_data/train_data_7.pq', 'train_data/train_data_8.pq', 'train_data/train_data_9.pq']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path train_data/train_data_0.pq
chunk_path train_data/train_data_1.pq
['train_data/train_data_0.pq', 'train_data/train_data_1.pq', 'train_data/train_data_10.pq', 'train_data/train_data_11.pq', 'train_data/train_data_2.pq', 'train_data/train_data_3.pq', 'train_data/train_data_4.pq', 'train_data/train_data_5.pq', 'train_data/train_data_6.pq', 'train_data/train_data_7.pq', 'train_data/train_data_8.pq', 'train_data/train_data_9.pq']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

...

chunk_path train_data/train_data_4.pq
chunk_path train_data/train_data_5.pq
['train_data/train_data_0.pq', 'train_data/train_data_1.pq', 'train_data/train_data_10.pq', 'train_data/train_data_11.pq', 'train_data/train_data_2.pq', 'train_data/train_data_3.pq', 'train_data/train_data_4.pq', 'train_data/train_data_5.pq', 'train_data/train_data_6.pq', 'train_data/train_data_7.pq', 'train_data/train_data_8.pq', 'train_data/train_data_9.pq']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path train_data/train_data_6.pq
chunk_path train_data/train_data_7.pq
['train_data/train_data_0.pq', 'train_data/train_data_1.pq', 'train_data/train_data_10.pq', 'train_data/train_data_11.pq', 'train_data/train_data_2.pq', 'train_data/train_data_3.pq', 'train_data/train_data_4.pq', 'train_data/train_data_5.pq', 'train_data/train_data_6.pq', 'train_data/train_data_7.pq', 'train_data/train_data_8.pq', 'train_data/train_data_9.pq']


Reading dataset with pandas:   0%|          | 0/2 [00:00<?, ?it/s]

chunk_path train_data/train_data_8.pq
chunk_path train_data/train_data_9.pq


In [6]:
data.head()

,id,pre_since_opened_0,pre_since_opened_1,pre_since_opened_2,pre_since_opened_3,pre_since_opened_4,pre_since_opened_5,pre_since_opened_6,pre_since_opened_7,pre_since_opened_8,...,is_zero_loans530,is_zero_loans3060,is_zero_loans6090,is_zero_loans90,is_zero_util,is_zero_over2limit,is_zero_maxover2limit,pclose_flag,fclose_flag,no_delinquencies
0,0,0,1,1,1,1,2,0,1,0,...,10,10,10,10,6,9,9,1,2,9
1,1,0,0,1,0,0,0,0,1,2,...,10,12,12,11,10,12,11,1,2,6
2,2,1,0,0,0,0,0,0,0,0,...,2,2,2,3,1,3,2,2,2,2
3,3,0,3,1,0,2,1,3,0,0,...,15,15,15,15,8,14,14,5,6,15
4,4,0,0,0,0,0,0,0,0,0,...,1,1,1,1,1,1,1,1,1,1


In [7]:
data.shape

(3000000, 398)

In [8]:
data.isna().sum().sum()

0

In [9]:
# Объединяем с целевой переменной
df = pd.merge(data, target_df[['id', 'flag']], on='id', how='left')
df = df.drop('id', axis=1)
print(f"Объединение файлов завершено.")
parquet_file_path = 'dataset_full_ohe.parquet'

# Преобразуем DataFrame в таблицу pyarrow
table = pa.Table.from_pandas(df)

# Сохраняем таблицу в формате Parquet
pq.write_table(table, parquet_file_path)
print(f"Сохранение файла завершено.")

Объединение файлов завершено.
Сохранение файла завершено.
